In [391]:
import cv2
import numpy as np

In [392]:
def detect_board(vid_frame_gray, ref_frame_gray, orb, bf):
    kp_board, des_board = orb.detectAndCompute(ref_frame_gray, None)
    kp_frame, des_frame = orb.detectAndCompute(vid_frame_gray, None)

    matches = bf.knnMatch(des_board, des_frame, k=2)

    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)

    if len(good) > 20:
        src_pts = np.float32([kp_board[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dst_pts = np.float32([kp_frame[m.trainIdx].pt for m in good]).reshape(-1,1,2)

        H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    else:
        return None
    h, w = ref_frame_gray.shape[:2]


    H_inv = np.linalg.inv(H)
    board_corners = np.float32([[0,0],[w,0],[w,h],[0,h]]).reshape(-1,1,2)
    camera_corners = cv2.perspectiveTransform(board_corners, H_inv)
    return camera_corners

def detect_cards(frame_gray,):
    frame_gray = cv2.GaussianBlur(frame_gray, (5,5), 0)
    frame_gray = cv2.adaptiveThreshold(
    frame_gray, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV,
    31, 7)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    frame_gray = cv2.morphologyEx(frame_gray, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(frame_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    test = cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(test, contours,contourIdx=-1, color=(0,0, 255), thickness=3, lineType=cv2.LINE_AA)
    cv2.imshow("Contours",test)

    cards = []

    for cnt in contours:
        if cv2.contourArea(cnt) < 5000:
             continue
        
        # Use minAreaRect for tilted boxes
        rect = cv2.minAreaRect(cnt) 
        cards.append(rect)
    return cards



In [393]:
board = cv2.imread("../data/board_reference.jpg")
board_gray = cv2.cvtColor(board, cv2.COLOR_BGR2GRAY)

BOARD_H, BOARD_W = board.shape[:2]

cap = cv2.VideoCapture("../data/easy2_mid.mp4")
cap.set(cv2.CAP_PROP_POS_FRAMES, 1800)
ret, frame = cap.read()
TABLE_H, TABLE_W = frame.shape[:2]


# frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

In [ ]:
orb = orb = cv2.ORB.create(
    nfeatures=500,
    scaleFactor=1.15,
    nlevels=10,
    fastThreshold=7
)
bf = cv2.BFMatcher(cv2.NORM_HAMMING)

In [ ]:
cv2.namedWindow("Tracking", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Tracking", 900, 600)
# cv2.namedWindow("Board View", cv2.WINDOW_NORMAL)
# cv2.resizeWindow("Board View", 900,600)

cv2.namedWindow("Contours", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Contours", 900,600)


i = 0
try:
    while True:
        ret, frame = cap.read()
        overlay = frame.copy()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if i % 30 == 0:
            board_corners_tmp = detect_board(frame_gray, board, orb, bf)

        if board_corners_tmp is not None: board_corners = board_corners_tmp
        
        pts = board_corners.astype(int)
        cv2.polylines(overlay, [pts], isClosed=True,
                    color=(0, 255, 0), thickness=3)
        H, _ = cv2.findHomography(board_corners, np.float32([[0,0],[BOARD_W,0],[BOARD_W,BOARD_H],[0,BOARD_H]]))
        board_view = cv2.warpPerspective(frame, H, (BOARD_W, BOARD_H))
            
            # table_view = cv2.warpPerspective(frame, H_table, (TABLE_W, TABLE_H))


            # table_view = cv2.warpPerspective(frame, H, (TABLE_W, TABLE_H))

        
        detected_cards = detect_cards(frame_gray)
        for rect in detected_cards:
            box = cv2.boxPoints(rect)
            box = np.int64(box)
            
            cv2.drawContours(overlay, [box], 0, (0, 255, 0), 2)


        i = (i+1) % 1000
        cv2.imshow("Tracking", overlay)
        # cv2.imshow("Board View", board_view)

        if cv2.waitKey(25) & 0xFF == 27:
            break
except Exception as e:
    cap.release()
    cv2.destroyAllWindows()
    raise e

cap.release()
cv2.destroyAllWindows()